# Описание проекта

Вам нужно защитить данные клиентов страховой компании «Хоть потоп». Разработайте такой метод преобразования данных, чтобы по ним было сложно восстановить персональную информацию. Обоснуйте корректность его работы.

Нужно защитить данные, чтобы при преобразовании качество моделей машинного обучения не ухудшилось. Подбирать наилучшую модель не требуется.

# Описание данных
Набор данных находится в файле `/datasets/insurance.csv`.
<br>**Признаки:** пол, возраст и зарплата застрахованного, количество членов его семьи.
<br>**Целевой признак:** количество страховых выплат клиенту за последние 5 лет.

# Содержание:

### [1. Загрузка данных](#1-bullet)
### [2. Умножение матриц](#2-bullet)
### [3. Алгоритм преобразования](#3-bullet)
### [4. Проверка алгоритма](#4-bullet)

## 1. Загрузка данных <a class="anchor" id="1-bullet"></a>

In [8]:
import pandas as pd
import numpy as np

from sklearn.linear_model import LinearRegression
from sklearn.model_selection import train_test_split
from sklearn.metrics import r2_score

In [9]:
df = pd.read_csv('/datasets/insurance.csv')
df.head(5)

,Пол,Возраст,Зарплата,Члены семьи,Страховые выплаты
0,1,41.0,49600.0,1,0
1,0,46.0,38000.0,1,1
2,0,29.0,21000.0,0,0
3,0,21.0,41700.0,2,0
4,1,28.0,26100.0,0,0


In [10]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 5000 entries, 0 to 4999
Data columns (total 5 columns):
Пол                  5000 non-null int64
Возраст              5000 non-null float64
Зарплата             5000 non-null float64
Члены семьи          5000 non-null int64
Страховые выплаты    5000 non-null int64
dtypes: float64(2), int64(3)
memory usage: 195.4 KB


In [11]:
df.describe()

,Пол,Возраст,Зарплата,Члены семьи,Страховые выплаты
count,5000.000000,5000.000000,5000.000000,5000.000000,5000.000000
mean,0.499000,30.952800,39916.360000,1.194200,0.148000
std,0.500049,8.440807,9900.083569,1.091387,0.463183
min,0.000000,18.000000,5300.000000,0.000000,0.000000
25%,0.000000,24.000000,33300.000000,0.000000,0.000000
50%,0.000000,30.000000,40200.000000,1.000000,0.000000
75%,1.000000,37.000000,46600.000000,2.000000,0.000000
max,1.000000,65.000000,79000.000000,6.000000,5.000000


In [12]:
for i in ['Возраст', 'Зарплата']:
    df[i] = df[i].astype(int)

In [13]:
df.duplicated().sum()

153

In [14]:
df.loc[df.duplicated() == True].sort_values('Возраст').head(20)

,Пол,Возраст,Зарплата,Члены семьи,Страховые выплаты
2429,1,18,39800,2,0
2512,1,19,43200,1,0
2269,1,19,43200,1,0
4129,1,19,35600,2,0
2853,0,19,51700,0,0
2694,1,19,52600,0,0
4935,1,19,32700,0,0
4726,1,19,31700,1,0
887,1,19,35500,0,0
3419,1,19,41600,1,0


Не готов утверждать, что это дубли. Оставим все данные.

## 2. Умножение матриц <a class="anchor" id="2-bullet"></a>

В этом задании вы можете записывать формулы в *Jupyter Notebook.*

Чтобы записать формулу внутри текста, окружите её символами доллара \\$; если снаружи —  двойными символами \\$\\$. Эти формулы записываются на языке вёрстки *LaTeX.* 

Для примера мы записали формулы линейной регрессии. Можете их скопировать и отредактировать, чтобы решить задачу.

Работать в *LaTeX* необязательно.

Обозначения:

- $X$ — матрица признаков (нулевой столбец состоит из единиц)

- $y$ — вектор целевого признака

- $P$ — матрица, на которую умножаются признаки

- $w$ — вектор весов линейной регрессии (нулевой элемент равен сдвигу)

Предсказания:

$$
a = Xw
$$

Задача обучения:

$$
w = \arg\min_w MSE(Xw, y)
$$

Формула обучения:

$$
w = (X^T X)^{-1} X^T y
$$

**Обоснование:**

Начальные данные:

$$
a = Xw
$$
$$
w = (X^T X)^{-1} X^T y
$$

---

Введем новую матрицу Z:

$$ Z = XP $$
$$ w_z = (Z^TZ)^{-1}Z^Ty$$

---

Подставим значения матрицы Z в предсказания:

$$a_z = Zw_z = XP((XP)^T(XP))^{-1}(XP)^Ty = XP((XP)^T)^{-1}(XP)^{-1}X^TP^Ty = XP(X^T)^{-1}(P^T)^{-1}X^{-1}P^{-1}X^TP^Ty = X((X^T X)^{-1} X^T y)(PP^{-1}(P^T)^{-1}P^T) = Xw*E*E=Xw = a$$

$$a_z = a$$

**Ответ: качество линейной регрессии не изменится.** 

---

$$ a_z = Zw_z = XP((XP)^T(XP))^{-1}(XP)^Ty $$
$$ a_z = XP(XP)^{-1}((XP)^T)^{-1}P^TX^Ty $$
$$ a_z = XPP^{-1}X^{-1}(P^TX^T)^{-1}P^TX^Ty $$
$$ a_z = XPP^{-1}X^{-1}(X^T)^{-1}(P^T)^{-1}P^TX^Ty $$

Т.к. матрица **P - обратимая**, то: $$PP^{-1} = E$$

Упрощаем выражение:

$$ a_z = XEX^{-1}(X^T)^{-1}EX^Ty $$


$$ a_z = XX^{-1}(X^T)^{-1}X^Ty $$
$$ a_z = X(X^TX)^{-1}X^Ty = Xw $$
$$ a_z = Xw = a$$

---

## 3. Алгоритм преобразования <a class="anchor" id="3-bullet"></a>

**Алгоритм**

1. Создаем случайную вспомогательную обратимую матрицу **Y**.
2. Умножаем матрицу признаков **X** на созданную матрицу **Y**.
3. В результате получаем закодированную матрицу **Z**.

**Обоснование**
1. На выходе данные в матрице **Z** закодированы.
2. Ключ шифрования - матрица **Y**.
3. Из доказанного выше знаем, что качество модели линейной регрессии не изменится.

## 4. Проверка алгоритма <a class="anchor" id="4-bullet"></a>

In [8]:
# Т.к. количество признаков (за исключением целевого) четыре, то и случайную матрицу сгенерируем размером 4*4.
num_rows = 4
num_columns = 4
random_matrix = np.random.random((num_rows, num_columns))

In [9]:
random_matrix

array([[0.35005248, 0.89780746, 0.06375912, 0.87956356],
       [0.1192968 , 0.06357553, 0.20698548, 0.42783222],
       [0.81444619, 0.36290057, 0.15774137, 0.43906125],
       [0.0396848 , 0.73089515, 0.64013029, 0.41067448]])

In [10]:
random_matrix_inv = np.linalg.inv(random_matrix)

In [11]:
(random_matrix_inv @ random_matrix).round()

array([[ 1.,  0., -0.,  0.],
       [-0.,  1.,  0., -0.],
       [ 0.,  0.,  1., -0.],
       [ 0., -0.,  0.,  1.]])

- Случайная матрица сгенерирована верно. Она обратима.

#### Построим 2 модели линейной регрессии (LinearRegression):
- для исходных данных
- для закодированных данных

In [12]:
df.columns

Index(['Пол', 'Возраст', 'Зарплата', 'Члены семьи', 'Страховые выплаты'], dtype='object')

Создадим модель для исходных (незакодированных) данных, обучим ее и рассчитваем значение метрики R2.

In [13]:
features = df.drop(['Страховые выплаты'], axis = 1)
target = df['Страховые выплаты']

In [14]:
# разделим данные features и target на две части (train и test) в соотношении 75% к 25% 
features_train, features_test, target_train, target_test = train_test_split(
    features, target, test_size=0.25, random_state=12345)

In [15]:
model = LinearRegression()
model.fit(features_train, target_train)
predictions = model.predict(features_test)

r2 = r2_score(predictions, target_test)
print('R2 =', r2)

R2 = -0.2869270135007398


Создадим модель для закодированных данных, обучим ее и рассчитваем значение метрики R2.

In [16]:
features_matrix = features.values
features_matrix_zip = features_matrix @ random_matrix

In [17]:
# разделим данные features и target на две части (train и test) в соотношении 75% к 25% 
features_train_zip, features_test_zip, target_train_zip, target_test_zip = train_test_split(
    features_matrix_zip, target, test_size=0.25, random_state=12345)

In [18]:
model = LinearRegression()
model.fit(features_train_zip, target_train_zip)
predictions_zip = model.predict(features_test_zip)
r2_zip = r2_score(predictions_zip, target_test_zip)
print('R2_zip =', r2_zip)

R2_zip = -0.2869270135018376


Сравним значение метрик двух моделей.

In [19]:
round(r2_zip - r2, 10)

-0.0

#### Вывод.
Значение метрик R2 для двух моделей практически одинаково. Качество модели линейной регрессии не изменилось. Алгорим кодирования данных может быть реализован.